# Import
本文档是汇总所有数据，尝试看看有没有什么特征是所有数据中都存在的

In [3]:
import os
import re
import sys 
from datetime import datetime
from loguru import logger



import torch
import numpy as np
from scipy.linalg import eig, svd, solve


import matplotlib.pyplot as plt
%matplotlib qt


import LFDRT



## Definition

In [10]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list


def Load_Single(ele_id, rootPath, DATA_SUFFIX):
    fd_pt = os.path.join(f"{rootPath}/{ele_id}_归档", DATA_SUFFIX, f"{ele_id}_{DATA_SUFFIX}.pt")
    if not os.path.exists(fd_pt):
        logger.warning(f"{fd_pt} does not exist")
        return None
    data_pt = torch.load(fd_pt, weights_only=False)
    _data_group = data_pt["data_group"]


    # DRT_list = []
    # EIS_list = []
    # # Iteration by channel
    # for j in _data_group['Channels']:
    #     _ch_data = _data_group[j]["chData"]
    #     _drt_data = _data_group[j]["DRTdata"]
    #     DRT_list.append(_drt_data)
    #     EIS_list.append(_ch_data)
    # logger.info(f"{len(DRT_list)}")
    # logger.info(f"{len(EIS_list)}")

    return _data_group
    

def DRT_Reconstruction_DRT(R_i, tau_i, f):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f  # Broadcasting tau_i to match f
    _RC = R_i[None, :] / (1+s[:,None] * tau_i[None,:])
    H = np.sum(_RC, axis=1)


    return H

In [5]:

rootPath = "D:/Baihm/EISNN/Archive/"
ele_list = SearchELE(rootPath)
DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Archive_New/"
# ele_list = SearchELE(rootPath)
# DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Invivo/"
# ele_list = SearchELE(rootPath, re.compile(r"(.+?)_Ver02"))
# DATASET_SUFFIX = "Outlier_Ver04"


n_ele = len(ele_list)
logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")

2025-07-14 16:05:39.326 | INFO     | __main__:<module>:15 - Search in D:/Baihm/EISNN/Archive/ and find 218 electrodes


## Load DRT Data

In [8]:
LoadDRTData_FLAG = False
if LoadDRTData_FLAG:
    DRT_SUFFIX = f"{DATASET_SUFFIX}_DRT_Ver01"
    DRT_list = []
    EIS_list = []

    for i in range(n_ele):
    # for i in range(0,1):
        fd_pt = os.path.join(ele_list[i][0], DRT_SUFFIX, f"{ele_list[i][1]}_{DRT_SUFFIX}.pt")
        if not os.path.exists(fd_pt):
            logger.warning(f"{fd_pt} does not exist")
            continue
        data_pt = torch.load(fd_pt, weights_only=False)
        _meta_group = data_pt["meta_group"]
        _data_group = data_pt["data_group"]


        ele_id  = _meta_group["ele_id"]
        elePath = _meta_group["elePath"]
        n_ch = _meta_group["n_ch"]      
        x_day_full = _meta_group["TimeSpan"]


        logger.warning(f"ELE[{i+1}/{n_ele}]: \t{ele_id} - {elePath}")


        # Iteration by channel
        for j in _data_group['Channels']:
            _ch_data = _data_group[j]["chData"]
            _drt_data = _data_group[j]["DRTdata"]
            DRT_list.append(_drt_data)
            EIS_list.append(_ch_data)
        logger.info(f"{len(DRT_list)}")
        logger.info(f"{len(EIS_list)}")
        

## Load EIS Data

In [6]:
if False:
    Org_EIS_list = []

    for i in range(n_ele):
    # for i in range(0,1):
        # logger.info(f"ELE Begin: {ele_list[i][0]}")
        fd_pt = os.path.join(ele_list[i][0], DATASET_SUFFIX, f"{ele_list[i][1]}_{DATASET_SUFFIX}.pt")
        if not os.path.exists(fd_pt):
            logger.warning(f"{fd_pt} does not exist")
            continue
        
        data_pt = torch.load(fd_pt)
        _meta_group = data_pt["meta_group"]
        _data_group = data_pt["data_group"]


        ele_id  = _meta_group["ele_id"]
        elePath = _meta_group["elePath"]
        n_ch = _meta_group["n_ch"]      
        x_day_full = _meta_group["TimeSpan"]


        logger.warning(f"ELE[{i+1}/{n_ele}]: \t{ele_id} - {elePath}")

        for j in _data_group['Channels']:
            logger.info(f"ELE[{ele_id}] - ch[{j:03d}] Begin") 
            channel_group_raw = _data_group[j]

            
            chData      = channel_group_raw['chData']         
            eis_seq     = channel_group_raw['eis_seq']        
            eis_cluster = channel_group_raw['eis_cluster']    
            eis_anomaly = channel_group_raw['eis_anomaly']    

            if chData.shape[2] != 5000:
                logger.error(f"ELE[{ele_id}] - ch[{j}] with less than 5000 samples")
                break
            
            chData = chData[eis_seq, :, :]

            Org_EIS_list.append(chData)
            



# Analysis

## DRT point & τ
该小节对所有DRT结果进行统计，需要之前执行过Load DRT Data才继续。
这里只是尝试对所有数据有一个大致判断，而且方向有点乱，后续不需要运行这段也行

In [ ]:
if LoadDRTData_FLAG:
    point_list = []
    tau_list = np.array([])
    for i in range(len(DRT_list)):
        logger.info(i)
        for j in range(len(DRT_list[i])):
            point_list.append(DRT_list[i][j].shape[1])
            point_list.append(DRT_list[i][j].shape[1])
            tau_list = np.concatenate([tau_list, DRT_list[i][j][2,:]])
    point_list = np.array(point_list)


2025-07-03 14:54:02.997 | INFO     | __main__:<module>:4 - 0
2025-07-03 14:54:02.998 | INFO     | __main__:<module>:4 - 1
2025-07-03 14:54:02.998 | INFO     | __main__:<module>:4 - 2
2025-07-03 14:54:02.998 | INFO     | __main__:<module>:4 - 3
2025-07-03 14:54:02.999 | INFO     | __main__:<module>:4 - 4
2025-07-03 14:54:02.999 | INFO     | __main__:<module>:4 - 5
2025-07-03 14:54:02.999 | INFO     | __main__:<module>:4 - 6
2025-07-03 14:54:03.000 | INFO     | __main__:<module>:4 - 7
2025-07-03 14:54:03.000 | INFO     | __main__:<module>:4 - 8
2025-07-03 14:54:03.000 | INFO     | __main__:<module>:4 - 9
2025-07-03 14:54:03.001 | INFO     | __main__:<module>:4 - 10
2025-07-03 14:54:03.001 | INFO     | __main__:<module>:4 - 11
2025-07-03 14:54:03.002 | INFO     | __main__:<module>:4 - 12
2025-07-03 14:54:03.002 | INFO     | __main__:<module>:4 - 13
2025-07-03 14:54:03.002 | INFO     | __main__:<module>:4 - 14
2025-07-03 14:54:03.003 | INFO     | __main__:<module>:4 - 15
2025-07-03 14:54:0

### $\tau$ Range

In [ ]:
if LoadDRTData_FLAG:
    def compute_probability(data, L, U):
        data = np.asarray(data)
        return np.mean((data > L) & (data < U))

    def find_bounds_for_p(data, target_p=0.9):
        data = np.sort(data)
        lower_q = (1 - target_p) / 2
        upper_q = 1 - lower_q
        L = np.quantile(data, lower_q)
        U = np.quantile(data, upper_q)
        return L, U

    p = compute_probability(np.log10(tau_list), L=-8, U=0)
    print(f"P(-8 < X < 2) = {p:.3f}")

    L, U = find_bounds_for_p(np.log10(tau_list), target_p=0.99)
    print(f"90% of data lies between L={L:.3f} and U={U:.3f}")

    fig = plt.figure()
    axis = fig.add_subplot(111)
    axis.hist(np.log10(tau_list),bins=10000, density=False)
    axis.set_xlabel(r"log($\tau$)")
    axis.set_ylabel(r"PDF")
    axis.set_title(r"Histogram for $\tau$")

### Point Distribution

In [ ]:
if LoadDRTData_FLAG:
    fig = plt.figure()
    axis = fig.add_subplot(111)
    # plt.plot(np.sort(point_list))
    # plt.semilogy(np.sort(tau_list))
    axis.hist(point_list, density=True)
    axis.set_xlabel(r"#Points")
    axis.set_ylabel(r"PDF")
    axis.set_title(r"Histogram for DRT Points")

Text(0.5, 1.0, 'Histogram for DRT Points')

### EIS Reconstruction

In [12]:
if LoadDRTData_FLAG:
    _drtData = DRT_list[0][1]
    _eisData = EIS_list[0][1]
    R_i = _drtData[0,:]
    tau_i = _drtData[2,:]
    freq_list = np.logspace(-1,10,1000)
    H = DRT_Reconstruction_DRT(R_i, tau_i, freq_list)
    plt.figure()
    plt.scatter(H.real, -H.imag)
    plt.scatter(_eisData[1,:], -_eisData[2,:])

## DRT_R VS DRT_C 
这里尝试抛开 τ, 分析更有意义的RC，但是也没看出什么苗头。

In [ ]:
ele_id = '06017758'
ch_id = 10

# ele_id = '09290511'
# ch_id = 4

Loe_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRT_Ver01")
Tik_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRTTik_Ver01")

Loe_DRT = Loe_Data[ch_id]['DRTdata']
Loe_EIS = Loe_Data[ch_id]['chData']
Tik_DRT = Tik_Data[ch_id]['DRTdata']
Tik_EIS = Tik_Data[ch_id]['chData']


fig = plt.figure()
axis1 = fig.add_subplot(121)
axis2 = fig.add_subplot(122)


cmap = plt.colormaps.get_cmap('rainbow_r')
for i in range(len(Loe_DRT)):
    # if i != 15: continue
    _loe = Loe_DRT[i]
    _tik = Tik_DRT[i]
    
    _color = cmap(i/len(Loe_DRT))

    axis1.scatter(_loe[0,:], 1/_loe[1,:], color=_color, s = 10, alpha=0.5)
    axis2.scatter(_tik[0,:], 1/_tik[1,:], color=_color, s = 10, alpha=0.5)
    # axis1.scatter(_loe[2,:], 1/_loe[1,:], color=_color, s = 10, alpha=0.5)
    # axis2.scatter(_tik[2,:], 1/_tik[1,:], color=_color, s = 10, alpha=0.5)


axis1.set_xscale('log')
axis1.set_yscale('log')
axis1.set_aspect('equal')
axis2.set_xscale('log')
axis2.set_yscale('log')
axis2.set_aspect('equal')


## Plot each DRT
fig = plt.figure()
_n = len(Loe_DRT)
_cols = 5
_rows = int(np.ceil(_n / _cols))

_xlim = (1e2, 1e8)
_ylim = (1e5, 1e12)

for i in range(len(Loe_DRT)):
    _loe = Loe_DRT[i]
    axis = fig.add_subplot(_rows, _cols, i + 1)
    
    _color = cmap(i/len(Loe_DRT))
    axis.scatter(_loe[0,:], 1/_loe[1,:], color=_color, s = 10, alpha=0.5)
    axis.set_xscale('log')
    axis.set_yscale('log')
    axis.set_xlim(_xlim)
    axis.set_ylim(_ylim) 



## Compare Tik & Loe
这里尝试对比Loewner 和 Tikhonov结果

In [ ]:
ele_id = '06017758'
ch_id = 10
Loe_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRT_Ver01")
Tik_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRTTik_Ver01")

Loe_DRT = Loe_Data[ch_id]['DRTdata']
Loe_EIS = Loe_Data[ch_id]['chData']
Tik_DRT = Tik_Data[ch_id]['DRTdata']
Tik_EIS = Tik_Data[ch_id]['chData']

day_id = -1

fig = plt.figure()
axis = fig.add_subplot(111)
# axis.stem(Loe_DRT[day_id][2,:], Loe_DRT[day_id][0,:], label='Loewner')
# axis.plot(Tik_DRT[day_id][2,:], Tik_DRT[day_id][0,:], '-.', label='Tikhonov')
# axis.stem(Loe_DRT[day_id][2,:], Loe_DRT[day_id][0,:], 'r', label='Loewner')
# axis.plot(Tik_DRT[day_id][2,:], Tik_DRT[day_id][0,:], 'r-.', label='Tikhonov')


# axis.stem(Loe_DRT[0][2,:], Loe_DRT[0][0,:], label='Loewner')
axis.plot(Tik_DRT[0][2,:], Tik_DRT[0][0,:], '-.', label='Tikhonov')
# axis.stem(Loe_DRT[-1][2,:], Loe_DRT[-1][0,:], 'r', label='Loewner')
axis.plot(Tik_DRT[-1][2,:], Tik_DRT[-1][0,:], 'r-.', label='Tikhonov')
axis.set_xscale('log')
axis.set_yscale('log')
axis.legend()




## Noise Interpolation

### Load Data

In [799]:
ele_id = '06017758'
ch_id = 10

# ele_id = '09290511'
# ch_id = 4


Loe_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRT_Ver01")
Tik_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRTTik_Ver01")
Org_data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}")


Loe_DRT = Loe_Data[ch_id]['DRTdata']
Loe_EIS = Loe_Data[ch_id]['chData']
Tik_DRT = Tik_Data[ch_id]['DRTdata']
Tik_EIS = Tik_Data[ch_id]['chData']

Org_EIS = Org_data[ch_id]['chData'][Org_data[ch_id]['eis_seq']]




if False:

    fig = plt.figure()
    axis1 = fig.add_subplot(121)
    axis2 = fig.add_subplot(122)

    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(Loe_DRT)):
        # if i != 15: continue
        _loe = Loe_DRT[i]
        _eis = Loe_EIS[i]
        
        _color = cmap(i/len(Loe_DRT))

        axis1.scatter(_loe[0,:], _loe[1,:], color=_color, s = 10, alpha=0.5)
        axis2.scatter(_eis[0,:], np.abs(_eis[1,:]+1j*_eis[2,:]), color=_color, s = 10, alpha=0.5)


    axis1.set_xscale('log')
    axis1.set_yscale('log')
    # axis1.set_aspect('equal')
    axis2.set_xscale('log')
    axis2.set_yscale('log')
    # axis2.set_aspect('equal')



    ## Plot each DRT
    fig = plt.figure()
    _n = len(Loe_DRT)
    _cols = 4
    _rows = int(np.ceil(_n / _cols))

    _xlim = (1e2, 1e8)
    _ylim = (1e5, 1e12)

    for i in range(len(Loe_DRT)):
        _loe = Loe_DRT[i]
        axis = fig.add_subplot(_rows, _cols, i + 1)
        
        _color = cmap(i/len(Loe_DRT))
        axis.scatter(_loe[0,:], 1/_loe[1,:], color=_color, s = 10)
        axis.set_xscale('log')
        axis.set_yscale('log')
        axis.set_xlim(_xlim)
        axis.set_ylim(_ylim) 






### Gaussian Monte-Carlo Sampleing

In [440]:
_day_id = 9

_org_loe = Loe_DRT[_day_id]
_org_eis = Loe_EIS[_day_id]


_f_org = _org_eis[0,:]
_Z_org = _org_eis[1,:] + 1j * _org_eis[2,:]
_f = _f_org
_Z = _Z_org



In [441]:
n_batch = 100
_drt_list = []


plt.figure()

_R_i, _C_i, _tau_i, _, _, _ = LFDRT.DRT_Analysis_Single(_f, _Z, REALFLAG=True)
_drt_list.append(np.array([_R_i, _C_i, _tau_i]))
plt.loglog(_tau_i,_R_i, '.')
# plt.loglog(_R_i,_C_i, '.')


for i in range(n_batch):
    _Z_noise = _Z + np.random.normal(0, 0.001, _Z.shape) * _Z.real + np.random.normal(0, 0.001, _Z.shape) * _Z.imag
    
    _R_i, _C_i, _tau_i, _, _, _ = LFDRT.DRT_Analysis_Single(_f, _Z_noise, REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))

    plt.loglog(_tau_i,_R_i, '.')
    # plt.loglog(_R_i,_C_i, '.')


### Subsampling 

In [643]:
_day_id = 4

_org_eis = Org_EIS[_day_id]

_f_org = _org_eis[0,:]
_Z_org = _org_eis[1,:] + 1j * _org_eis[2,:]
_f = _f_org
_Z = _Z_org


n_batch = 50
freq_base = 1000

_f = [np.take(_f, indices=range(i, 5000, n_batch), axis=0) for i in range(freq_base, freq_base+n_batch)]
_Z = [np.take(_Z, indices=range(i, 5000, n_batch), axis=0) for i in range(freq_base, freq_base+n_batch)]

_f = np.stack(_f,axis=0)
_Z = np.stack(_Z,axis=0)

if False:
    plt.figure()
    for i in range(_f.shape[0]):
        plt.loglog(_f[i,:], np.abs(_Z[i,:]))


In [646]:
_drt_list = []

fig = plt.figure()
axis = fig.add_subplot(111)
for i in range(_f.shape[0]):
    # if i <20: continue
    _R_i, _C_i, _tau_i, _, _, _ = LFDRT.DRT_Analysis_Single(_f[i,:], _Z[i,:], REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))

    # axis.loglog(_tau_i,_R_i, '.')
    # axis.loglog(_R_i,_C_i, '.')

    
    axis.scatter(_R_i[1:-4], _C_i[1:-4], s=2, color = 'gray')
    axis.scatter(_R_i[-1], _C_i[-1], s=2, color = 'red')
    axis.scatter(_R_i[0], _C_i[0], s=2, color = 'orange')

# axis.set_aspect('equal')

axis.set_xscale('log')
axis.set_yscale('log')

### Bootstrap

In [853]:

def stratified_subsample(total_size, n_subsample, idx_base=1000, seed=None):
    """
    将 total_size 个点均匀分成 n_subsample 段，每段随机取 1 个索引，无放回采样。
    """
    if seed is not None:
        np.random.seed(seed)
    
    indices = np.arange(idx_base, total_size)
    bins = np.array_split(indices, n_subsample)
    sampled_indices = [np.random.choice(bin, 1)[0] for bin in bins]
    
    return np.array(sampled_indices)

In [854]:
_day_id = 9

_org_eis = Org_EIS[_day_id]


_f_org = _org_eis[0,:]
_Z_org = _org_eis[1,:] + 1j * _org_eis[2,:]
_f = _f_org
_Z = _Z_org


n_batch = 50
n_point = 100

_f_bootstrap = []
_Z_bootstrap = []
for i in range(n_batch):
    _idx = stratified_subsample(_f.shape[0], n_point)
    _f_bootstrap.append(_f[_idx])
    _Z_bootstrap.append(_Z[_idx])

_f = np.stack(_f_bootstrap, axis=0)
_Z = np.stack(_Z_bootstrap, axis=0)

if False:
    plt.figure()
    for i in range(_f.shape[0]):
        plt.loglog(_f[i,:], np.abs(_Z[i,:]))


In [855]:
_drt_list = []

fig = plt.figure()
axis = fig.add_subplot(111)
for i in range(_f.shape[0]):
    # if i <20: continue
    _R_i, _C_i, _tau_i, _, _, _ = LFDRT.DRT_Analysis_Single(_f[i,:], _Z[i,:], REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))

    # axis.scatter(_tau_i,_R_i, s=2, color = 'gray')
    # axis.scatter(_R_i, _C_i, s=2, color = 'gray')
    # axis.scatter(_tau_i[0],_R_i[0], s=2)
    # axis.scatter(_tau_i[0],_R_i[0]*_R_i.shape[0], s=2)

    ## Last Point Discuss
    axis.scatter(_tau_i[1:-1],_R_i[1:-1], s=2, color = 'gray')
    axis.scatter(_tau_i[0],_R_i[0], s=2, color = 'orange')
    axis.scatter(_tau_i[-1],_R_i[-1], s=2, color = 'red')
    # axis.scatter(_tau_i[-4],_R_i[-4], s=2, color='blue')
    # axis.scatter(_tau_i[-3],_R_i[-3], s=2, color='green')
    # axis.scatter(_tau_i[-2],_R_i[-2], s=2, color='orange')
    # axis.scatter(_tau_i[-1],_R_i[-1], s=2, color='red')

    ## Order Correction Discuss - Conclusion: correction should NOT be applied here
    # axis.scatter(_tau_i[1:-10],_R_i[1:-10], s=2, color='red')
    # axis.scatter(_tau_i[1:-10],_R_i[1:-10]*_R_i.shape[0], s=2, color='blue')

    ## RC Discussion
    # axis.scatter(_R_i[1:-4], _C_i[1:-4], s=2, color = 'gray')
    # axis.scatter(_R_i[-1], _C_i[-1], s=2, color = 'red')
    # axis.scatter(_R_i[0], _C_i[0], s=2, color = 'orange')


    # axis.scatter(_R_i*_R_i.shape[0], 1/_C_i*_R_i.shape[0], s=2, color = 'gray')
    # axis.scatter(_R_i*_R_i.shape[0], 1/_C_i*_R_i.shape[0], s=2)

# axis.set_xlim([1e-10,1e2])
# axis.set_ylim([1e0,1e10])
axis.set_xscale('log')
axis.set_yscale('log')
axis.set_aspect('equal')

### Outlier Detection

In [836]:
from statsmodels.nonparametric.smoothers_lowess import lowess
import numpy as np
import matplotlib.pyplot as plt



_R_i = np.concatenate([i[0,:] for i in _drt_list])
# _R_i = np.concatenate([i[0,:]*i[0,:].shape[0] for i in _drt_list])
_C_i = np.concatenate([i[1,:] for i in _drt_list])
_tau_i = np.concatenate([i[2,:] for i in _drt_list])

_log_R = np.log10(_R_i)
_log_tau = np.log10(_tau_i)


# 假设已有原始 x, y 数据
x_valid = _tau_i
y_valid = _R_i

x_log = np.log10(x_valid)
y_log = np.log10(y_valid)

# 做 robust LOESS
# y_log_smooth = lowess(y_log, x_log, frac=0.05, it=3, return_sorted=False)
y_log_smooth = lowess(2*y_log-x_log, x_log, frac=0.05, it=3, return_sorted=False)

R_loess = np.power(10, y_log_smooth)
tau_loess = _tau_i



if True:
    plt.figure()
    # plt.scatter(x_log, y_log, color='gray', s=1, label='Raw log-log')
    # plt.scatter(x_log, y_log_smooth, color='red', s=1, label='LOESS fit (log-log)')

    plt.scatter(x_log, 2*y_log-x_log, color='gray', s=1, label='Raw log-log')
    # plt.scatter(x_log, y_log_smooth, color='red', s=1, label='LOESS fit (log-log)')
    plt.plot(x_log, y_log_smooth, color='red', label='LOESS fit (log-log)')

    # plt.scatter(x_log[1:-1], 2*y_log[1:-1]-x_log[1:-1], color='gray', s=1, label='Raw log-log')


    # plt.plot(x_log, y_log_smooth, color='red', label='LOESS fit (log-log)')
    # plt.scatter(x_log[x_log<-2], np.power(10,y_log)[x_log<-2], s=10, label='Raw log-log')
    # plt.scatter(x_log[x_log<-2], np.power(10,y_log_smooth)[x_log<-2], color='red', label='LOESS fit (log-log)')
    plt.xlabel('log10(x)')
    plt.ylabel('log10(y)')
    
    # plt.xlim([-10,2])
    # plt.ylim([0,10])
    plt.legend()
    plt.title('LOESS in log-log space')
    plt.show()


In [809]:
fig = plt.figure(figsize=(15, 4))
axis1 = fig.add_subplot(141)
axis2 = fig.add_subplot(142)
axis3 = fig.add_subplot(143)
axis4 = fig.add_subplot(144)

axis1.set_title(r"R vs. $\tau$")
axis1.scatter(_tau_i, _R_i, s=2, color='gray')
axis2.set_title(r"1/C vs. $\tau$")
axis2.scatter(_tau_i, 1/_C_i, s=2, color='gray')
axis3.set_title(r"R/C vs. $\tau$")
axis3.scatter(_tau_i, _R_i/_C_i, s=2, color='gray')
axis4.set_title(r"C vs. R")
axis4.scatter(_R_i, _C_i, s=2, color='gray')

axis1.set_xscale('log')
axis1.set_yscale('log')
axis2.set_xscale('log')
axis2.set_yscale('log')
axis3.set_xscale('log')
axis3.set_yscale('log')
axis4.set_xscale('log')
axis4.set_yscale('log')




In [810]:

Loe_H_LOESS = DRT_Reconstruction_DRT(np.power(10, y_log_smooth), _tau_i, _f_org[1000:])
Loe_H_org = DRT_Reconstruction_DRT(_R_i, _tau_i, _f_org[1000:])
# Loe_H_LOESS = DRT_Reconstruction_DRT(np.power(10, y_log_smooth), _tau_i, _f[0,:])
# Loe_H_org = DRT_Reconstruction_DRT(_R_i, _tau_i, _f[0,:])
# Loe_H_LOESS = DRT_Reconstruction_DRT(np.power(10, y_log_smooth), _tau_i, _f)
# Loe_H_org = DRT_Reconstruction_DRT(_R_i, _tau_i, _f)
plt.figure()


# plt.scatter(_Z_org[1000:].real, -_Z_org[1000:].imag, s=2, label='Org')
# plt.scatter(_Z[0,:].real, -_Z[0,:].imag, s=10, label='Org')
# plt.scatter(_Z.real, -_Z.imag, s=10, label='Org')
# plt.scatter(Loe_H_org.real/n_batch, -Loe_H_org.imag/n_batch, s=10, label='Loe DRT')
# plt.scatter(Loe_H_LOESS.real/n_batch, -Loe_H_LOESS.imag/n_batch, s=10, label='LOESS DRT')
# plt.axis('equal')

# plt.loglog(_f, np.abs(_Z), '.', label='Org')
# plt.scatter(_f_org[1000:], np.abs(_Z_org[1000:]), s=1, label='Org')
plt.scatter( _f_org[1000:], np.abs(Loe_H_org)/n_batch, s=2, label='Loe DRT')
plt.scatter( _f_org[1000:], np.abs(Loe_H_LOESS)/n_batch, s=2, label='LOESS DRT')
plt.scatter(_f_org[1000:], np.abs(_Z_org[1000:]), s=2, label='Org')
plt.xscale('log')
plt.yscale('log')


# plt.semilogx(_f, -180/np.pi*np.angle(_Z), '.', label='Org')
# plt.semilogx(_f, -180/np.pi*np.angle(Loe_H_org), '.', label='Loe DRT')
# plt.semilogx(_f, -180/np.pi*np.angle(Loe_H_LOESS), '.', label='LOESS DRT')


plt.legend()

In [811]:
def detect_loess_outliers(y, y_loess, threshold=3.0):
    """
    标记LOESS拟合后的离群点。
    
    参数：
    - y: 原始纵轴数据
    - y_loess: LOESS拟合后的结果
    - threshold: 离群点判断阈值倍数，默认 3 × MAD
    
    返回：
    - outlier_mask: bool数组，True表示离群点
    - residual: 每个点的残差 y - y_loess
    - mad: 残差的中位数绝对偏差
    """
    y = np.asarray(y)
    y_loess = np.asarray(y_loess)
    residual = y - y_loess
    mad = np.median(np.abs(residual - np.median(residual)))
    
    if mad == 0:
        # 全部点过于集中，无法用 MAD 判断
        outlier_mask = np.zeros_like(residual, dtype=bool)
    else:
        outlier_mask = np.abs(residual) > threshold * mad
    
    return outlier_mask, residual, mad


In [812]:
# outlier_mask, residual, mad = detect_loess_outliers(y_log, y_log_smooth, threshold=5)
# # _a = x_log[outlier_mask]
# # _b = y_log[outlier_mask]
# _a = x_log[~ outlier_mask]
# _b = y_log[~ outlier_mask]
# plt.figure()
# plt.scatter(_a, _b, s=2)

### Imag Real Analysis
* 之前选择把看着不是纯实数的解都扔掉了，这里好奇加回来会不会更好，所以这里记录一下

In [464]:
_day_id = 9

_org_eis = Org_EIS[_day_id]


_f_org = _org_eis[0,:]
_Z_org = _org_eis[1,:] + 1j * _org_eis[2,:]
_f = _f_org
_Z = _Z_org


n_batch = 100
n_point = 100

_f_bootstrap = []
_Z_bootstrap = []
for i in range(n_batch):
    _idx = stratified_subsample(_f.shape[0], n_point)
    _f_bootstrap.append(_f[_idx])
    _Z_bootstrap.append(_Z[_idx])

_f = np.stack(_f_bootstrap, axis=0)
_Z = np.stack(_Z_bootstrap, axis=0)

if False:
    plt.figure()
    for i in range(_f.shape[0]):
        plt.loglog(_f[i,:], np.abs(_Z[i,:]))


In [470]:
_drt_list = []

fig = plt.figure()
axis = fig.add_subplot(111)
for i in range(_f.shape[0]):
    # if i <20: continue
    _R_i, _C_i, _tau_i, _, _, _ = LFDRT.DRT_Analysis_Single(_f[i,:], _Z[i,:], REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))

    print(_R_i.shape[0])

    axis.scatter(_tau_i,_R_i, s=2, color = 'gray')
    # axis.loglog(_R_i,_C_i, '.')
axis.set_xlim([1e-10,1e2])
axis.set_ylim([1e0,1e10])
axis.set_xscale('log')
axis.set_yscale('log')
axis.set_aspect('equal')

18
14
12
14
12
12
14
16
16
16
14
10
14
14
14
12
10
12
14
16
12
16
14
14
14
14
14
18
16
14
14
12
16
14
12
16
16
14
12
18
18
16
14
16
16
14
12
14
14
14
14
16
16
14
16
14
16
14
12
16
14
14
14
10
12
14
14
14
16
14
14
14
14
14
14
12
16
16
16
12
14
14
14
12
12
16
12
16
14
12
12
18
12
14
14
14
14
14
12
14


### Smith Circuit

In [671]:


# 你的输入数据
# f: 频率数组 (shape: (N,))
# Z: 复阻抗数组 (shape: (N,))
# R_list, C_list: 并联 RC 参数列表

# 特征阻抗 Z0（根据实际系统选定，一般 50 Ω 或 1 Ω）
Z = _Z[0,:]
f = _f[0,:]
Z0 = np.sqrt(np.min(np.abs(Z)) * np.max(np.abs(Z)))
# 计算反射系数 Gamma
Gamma = (Z - Z0) / (Z + Z0)

# 同理计算 RC 模型
# omega = 2 * np.pi * f
# Z_model = sum(R / (1 + 1j*omega*R*C) for R, C in zip(R_list, C_list))
# Gamma_model = (Z_model - Z0) / (Z_model + Z0)

# 提取极坐标
theta = np.angle(Gamma)
r = np.abs(Gamma)

# theta_m = np.angle(Gamma_model)
# r_m = np.abs(Gamma_model)

# 绘图
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(6,6))
# 原始测量点
ax.plot(theta, r, '.', label='Measured')
# RC 模型
# ax.plot(theta_m, r_m, 'r--', label='RC model')

# 美化：画单位圆与坐标网格
# ax.set_rmax(1.0)
ax.grid(True)
ax.set_title('Smith Circle (Polar Plot of Γ)', va='bottom')
ax.legend(loc='lower right')

plt.show()


## Peak Analysis
目前得到了比较接近真实状态的DRT结果，但是反应峰值和CPE部分缠在一起了，需要想办法分开

### Differential

In [163]:
# R_loess, tau_loess
loess_drt = np.stack([R_loess, tau_loess])
loess_drt = loess_drt[:,loess_drt[1,:].argsort()]
loess_drt_log = np.log(loess_drt)

plt.figure()
plt.scatter(loess_drt[1,:], loess_drt[0,:], s=2, label='LOESS DRT')
# plt.plot(loess_drt[1,:], loess_drt[0,:], label='LOESS DRT')
plt.xscale('log')
plt.yscale('log')

In [164]:
_loess_diff = np.diff(loess_drt_log, axis=1)

plt.figure()
plt.scatter(loess_drt_log[1,1:], _loess_diff[0,:]/_loess_diff[1,:], s=2, label='LOESS DRT')
# plt.plot(LOESS_DRT[1,:], LOESS_DRT[0,:], label='LOESS DRT')
# plt.xscale('log')
# plt.yscale('log')
